# Régression Linéaire pour la prédiction des prix de voiture

Modèle simple pour prédire les prix en fonction du kilométrage.

## Définitions :

- **Normalisation** : Redimensionner les variables numériques pour qu'elles soient comparables sur une échelle commune.
- **Machine Learning** : Donner à une machine la capacité d'apprendre sans la programmer de façon explicite.
- **Apprentissage supervisé** : Technique d'apprentissage la plus courante en machine learning. On donne des exemples à la machine qu'elle doit étudier pour en créer des modèles.
- **Dataset** : Ensemble d'exemples données à la machine pour apprendre (tableau de données).
- **Problème de regression** : On cherche à prédire la valeur d'une variable continue, c'est-à-dire une variable qui peux prendre une infinité de valeurs.
- **Problème de classification** : On cherche à prédire la valeur d'une variable discrète, c'est-à-dire une variable qui prends certaines valeurs.

## Prérequis :

Pour travailler de manière plus optimale, l'utilisation de matrice est fortement recommandée. Pour ce projet, j'ai décidé de créer moi-même une classe `Matrix` contenant toutes les opérations matricielles. Il est en revanche recommandé d'utiliser des librairies externes, telles que `numpy` ou `pandas` pour plus de facilité.

In [31]:
class Matrix(object):
    values: list[list[float]]

    def __init__(self, values: list[list[float]]):
        self.values = values

    def shape(self) -> tuple[int, int]:
        return len(self.values), len(self.values[0])

    def min(self) -> float:
        return min(min(value) for value in self.values)

    def max(self) -> float:
        return max(max(value) for value in self.values)

    def sum(self) -> float:
        return sum(sum(row) for row in self.values)

    def substract(self, matrix: list[list[float]]) -> list[list[float]]:
        return [[i - j for i, j in zip(*m)] for m in zip(self.values, matrix)]

    def multiply(self, matrix: list[list[float]]) -> list[list[float]]:
        if len(self.values[0]) != len(matrix):
            raise ValueError("The number of columns in the first matrix must correspond to the number of rows in the second matrix.")
        return [[sum(m * n for m, n in zip(i, j)) for j in zip(*matrix)] for i in self.values]

    def transpose(self) -> list[list[float]]:
        return list(map(list, zip(*self.values)))

    def scalar(self, n: float) -> list[list[float]]:
        return [[x * n for x in row] for row in self.values]

    def mean(self) -> float:
        shape: tuple[int, int] = self.shape()
        return self.sum() / (shape[0] * shape[1])

    def square(self) -> list[list[float]]:
        shape: tuple[int, int] = self.shape()
        return [[self.values[i][j] ** 2 for j in range(shape[1])] for i in range(shape[0])]


## Étape 1 : Dataset

La première étape consiste à récupérer les données de notre dataset. On va extraire deux matrices qui correspondent à notre target et nos features.
Dans notre situation, nous avons donc un vecteur target (prix) et une matrice features (kilométrages).

Voici une représentation de nos matrices :

$$
\vec{x} = \begin{pmatrix} 240000 \cr 139800 \cr \ldots \cr 61789 \cr \end {pmatrix}
$$

$$
\vec{y} = \begin{pmatrix} 3650 \cr 3800 \cr \ldots \cr 8290 \cr \end {pmatrix}
$$

Par convention, on note $m$ le nombre d'exemple que l'on a dans notre dataset et on note $n$ le nombre de features que l'on a dans notre dataset. Notre matrice target a une taille de $(m, 1)$ et notre matrice features a une taille de $(m, n)$. Avec notre dataset, nous avons donc $m = 24$ et $n = 1$.

Pour ce faire, voici une classe permettant d'extraire les données (à l'initialisation de notre classe) de notre dataset (notre fichier CSV) en Python.

In [32]:
from csv import reader

class Dataset(object):
    target: Matrix
    features: Matrix

    def __init__(self, path: str):
        self.target = Matrix([])
        self.features = Matrix([])
        self.__read_dataset(path)

    def __read_dataset(self, path: str) -> None:
        try:
            with open(path, mode="r", newline="") as file:
                r = reader(file)
                next(r)
                for row in r:
                    self.target.values.append([float(row[1])])
                    self.features.values.append([float(row[0])])
        except FileNotFoundError:
            raise RuntimeError(f"The file {path} does not exist.")
        except Exception as e:
            raise RuntimeError(f"An error occurred while reading the CSV file: {e}")

![Raw Data Visualisation](images/raw_data.png)
![Dataset representation](images/dataset.png)

## Étape 2 : Modèle

À partir du dataset (et de la représentation graphique présente plus haut), on peux visualiser un nuage de points. Pour ce projet, nous allons utiliser un modèle linéaire, cependant il existe d'autres type de modèle disponible.

Il est important de noter que c'est nous qui décidons de quel modèle la machine doit utiliser et c'est la machine qui doit apprendre les paramètres. Le modèle est une généralisation de l'ensemble des points de notre dataset, un bon modèle est un modèle qui nous donne les plus petites erreurs.

Pour notre modèle linéaire, nous avons donc la formule suivante :

$$
f(x)=ax+b
$$

La formule sous forme matricielle :

$$
F = X \cdot \theta
$$

Ce modèle a deux paramètres $a$ et $b$ (cf. coefficients polynome). Comme c'est la machine qui doit apprendre les paramètres, nous utiliserons au lancement du programme des paramètres aléatoire.

Voici des fonctions permettant de calculer notre model :

In [33]:
def model(self, x: float, theta: Matrix) -> float:
    return theta.values[0][0] + (theta.values[1][0] * x)

def matrix_model(self, x: Matrix, theta: Matrix) -> Matrix:
    return Matrix(x.multiply(theta.values))

## Étape 3 : Fonction Coût

La fonction coût (cf. Fonction Quadratique Moyenne) permet de calculer le coût entre le modèle qu'elle est en train de développer et les vraies valeur de target. Trouver le minimum de la fonction coût revient à trouver le meilleur modèle pour notre programme.

La fonction va mesurer la distance entre la prédiction (le point sur notre droite) et la valeur réelle (cf. norme euclidienne), c'est ce que l'on nomme erreur. On va rassembler toutes les erreurs dans une fonction nommé $J$.

Pour notre fonction coût, la formule est la suivate :

$$
J(a, b) = \frac{1}{2m} \displaystyle\sum_{i=1}^m (f(x^i) - y^i)^2
$$

La formule sous forme matricielle :

$$
J(\theta) = \frac{1}{2m} \displaystyle\sum_{i=1}^m (X \cdot \theta - Y)^2
$$

![Cost function](images/cost.png)

![Error model](images/model_error.png)

## Étape 4 : Algorithme de minimisation

L'algorithme de minimisation est une stratégie qui cherche à trouver quels sont les paramètres de notre modèle qui minimise la fonction coût, c'est-à-dire qui minimise l'ensemble de nos erreurs. Pour notre projet, nous utiliserons la déscente de gradient. La déscente de gradient est un algorithme d'optimisation qui converge vers le minimum d'une fonction convexe.

Dans notre situation, la fonction coût à la même allure qu'une fonction carré (car on fait la somme de carré), c'est-à-dire une allure parabolique. Sur cette fonction, on recherche le minimum de $J$ par rapport à $a$. Pour ce faire, on choisi au hasard un point sur la courbe $J$, on va mesurer sa dérivé et on va aller dans la direction de la pente qui déscends. L'hyper-paramètre $\alpha$ correspond à notre vitesse de convergence (aka. learning_rate).

Pour notre déscente de gradient, voici les formules :

$$
a = a - \alpha \frac{\partial J(a, b)}{\partial a}
$$

$$
b = b - \alpha \frac{\partial J(a, b)}{\partial b}
$$

$$
\theta = \theta - \alpha \frac{\partial J(\theta)}{\partial \theta}
$$

Pour calculer les dérivés de $a$ et $b$ par rapport à $J$, on utilise les formules suivantes :

$$
\frac{\partial J(a, b)}{\partial a} = \frac{1}{m} \displaystyle\sum_{i=1}^m x^i(ax^i + b - y^i)
$$

$$
\frac{\partial J(a, b)}{\partial b} = \frac{1}{m} \displaystyle\sum_{i=1}^m 1(ax^i + b - y^i)
$$

$$
\frac{\partial J(\theta)}{\partial \theta} = \frac{1}{m} X^T (X \cdot \theta - Y)
$$

Voici un exemple de code pour la déscente de gradient :

In [ ]:
def gradient(self, x: Matrix, y: Matrix, theta: Matrix) -> Matrix:
    m: int = x.shape()[0]
    predictions: Matrix = self.matrix_model(x, theta)
    errors: Matrix = Matrix(predictions.substract(y.values))
    x_t: Matrix = Matrix(x.transpose())
    grad: Matrix = Matrix(x_t.multiply(errors.values))
    return Matrix([[element * (1 / m) for element in row] for row in grad.values])

def gradient_descent(self, x: Matrix, y: Matrix, theta: Matrix) -> Matrix:
    for i in range(self.iterations):
        gradient: Matrix = self.gradient(x, y, theta)
        updated_theta: Matrix = Matrix(gradient.scalar(self.learning_rate))
        theta: Matrix = Matrix(theta.substract(updated_theta.values))
    return theta

![Square function](images/squared_function.png)
![Regressions](images/regression.png)